In [1]:
%matplotlib inline
import pandas as pd

In [2]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [3]:
sales1 = pd.read_csv('sales1.csv')
sales1

,Book title,Number sold,Sales price,Royalty paid
0,The Bricklayer’s Bible,8,2.99,0.55
1,Swimrand,2,1.99,0.35
2,Pining For The Fisheries of Yore,28,2.99,0.55
3,The Duck Goes Here,34,2.99,0.55
4,The Tower Commission Report,4,11.50,4.25


In [4]:
sales2 = pd.read_csv('sales2.csv')
sales2.fillna('')

,Title,Units sold,List price,Royalty
0,,,,
1,Sales report for Q4,,,
2,E-Book Reader US Store,,,
3,Pining for the Fisheries of Yore,80,3.5,14.98
4,Swimrand,1,2.99,0.14
5,The Bricklayer's Bible,17,3.5,5.15
6,The Duck Goes Here,34,2.99,5.78
7,The Tower Commission Report,4,9.5,6.2
8,US royalties (USD),,,32.25
9,,,,


### Challenge: first combine these sales together into a single dataframe, then compute how much money consumers spent on each book in each currency.

First, let's rename the columns in ```sales1``` and ```sales2```.  This will make it easier for us to join, merge, slice, and dice later.

In [5]:
sales1 = sales1.rename(columns={'Book title': 'title',
                                'Number sold':'number', 
                                'Sales price':'they_paid', 
                                'Royalty paid':'we_got'})
sales2 = sales2.rename(columns={'Title': 'title',
                                'Units sold':'number', 
                                'List price':'they_paid', 
                                'Royalty':'we_got'})

My original idea was to break up the ```sales2``` ```DataFrame``` by currency and add a column of the appropriate name manually.  It turns out there's a slicker way to do that.

This command might take a little explaining.  We're finding the names of the currencies by finding (string) values in the ```'title'``` column of ```sales2``` which end with a close-paren, splitting those strings on spaces into lists of strings, extracting the final elements from those lists, and stripping the parentheses from the extracted elements.

In [6]:
currency = sales2['title'].where(sales2['title'].str.endswith(')')).str.split().str[-1].str.strip('()')

Add a ```'currency'``` column to ```sales2```, then delete any rows which don't contain an actual book title.  Let's do this by finding and eliminating rows with ```NaN``` values in the ```'number'``` column.

In [7]:
sales2['currency'] = currency.fillna(method='bfill')
sales2 = sales2[sales2['number'].notnull()]

Add a ```'currency'``` column to ```sales1```

In [8]:
sales1.insert(len(sales1.columns), 'currency', 'USD')

Here's the implementation of my original idea.  It does also work, but because of ```sales2```'s formatting, it's a bit less elegant than the work above.

In [9]:
## Let's break up the sales2 DataFrame 
## Remember that the final index on a call to loc is inclusive
#us_sales = sales2.loc[3:7]
#uk_sales = sales2.loc[13:15]
#france_sales = sales2.loc[21:22]

## Compute the number of columns of each DataFrame
## This will allow us to insert the 'currency' column in the last position
#num_sales1_cols = len(sales1.columns)
#num_sales2_cols = len(sales2.columns)

## Insert the columns
## This is an in-place operation
#sales1.insert(num_sales1_cols, 'currency', 'USD')
#us_sales.insert(num_sales2_cols, 'currency', 'USD')
#uk_sales.insert(num_sales2_cols, 'currency', 'GBP')
#france_sales.insert(num_sales2_cols, 'currency', 'EUR')

## Concatenate the sales DataFrames
#sales = pd.concat([sales1, us_sales, uk_sales, france_sales])

Concatenate the ```sales``` ```DataFrame```s

In [10]:
sales = pd.concat([sales1, sales2])

Insert a ```total_paid``` column into the ```sales``` ```DataFrame```

In [11]:
sales.insert(len(sales.columns)-1, 'total_paid', sales['number']*sales['they_paid'])

Now, we compute the total consumer spending for each book by currency

In [12]:
total_sales = sales.groupby(['title','currency'])[['total_paid']].sum()

In [13]:
total_sales

total_paid
title                            currency            
Pining For The Fisheries of Yore USD            83.72
Pining for the Fisheries of Yore GBP           140.53
                                 USD           280.00
Swimrand                         EUR            15.92
                                 USD             6.97
The Bricklayer's Bible           GBP            50.83
                                 USD            59.50
The Bricklayer’s Bible           USD            23.92
The Duck Goes Here               EUR            23.88
                                 USD           203.32
The Tower Commission Report      GBP            26.00
                                 USD            84.00